In [2]:
import boto3

region = boto3.session.Session().region_name 

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-processing-container'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)


In [3]:
from sagemaker import get_execution_role
role = get_execution_role()
role

'arn:aws:iam::064258348567:role/service-role/AmazonSageMaker-ExecutionRole-20200131T104492'

In [4]:
processing_repository_uri

'064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-processing-container:latest'

In [10]:
!docker build -t $ecr_repository .

Sending build context to Docker daemon  201.7kB
Step 1/8 : FROM python:3.7-slim-buster
 ---> b142ab477505
Step 2/8 : MAINTAINER ASC "mbod@asc.upenn.edu jmparelman@gmail.com"
 ---> Using cache
 ---> 9af566ff1024
Step 3/8 : RUN apt-get -y update && apt-get install -y --no-install-recommends         build-essential         python-dev         python3-pip     && pip3 install -U pip
 ---> Using cache
 ---> 988639001443
Step 4/8 : ENV SPACY_VERSION 3.0.5
 ---> Using cache
 ---> 84ce7cf699fe
Step 5/8 : RUN     pip3 install -U pandas          tqdm          gensim          sklearn     && pip3 install -U spacy==${SPACY_VERSION}     && python3 -m spacy download en_core_web_sm
 ---> Using cache
 ---> 73243f0fe6bb
Step 6/8 : RUN mkdir -p /opt/ml/processing/input/speeches /opt/ml/processing/input/descr /opt/ml/processing/output
 ---> Running in 6cef0c15479a
Removing intermediate container 6cef0c15479a
 ---> 0f1c03c2f52b
Step 7/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Running in 961632d7cec2
Removing inter

In [17]:
account_id

'064258348567'

In [18]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 064258348567.dkr.ecr.us-east-1.amazonaws.com


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [19]:
!aws ecr create-repository --repository-name $ecr_repository



An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sagemaker-processing-container' already exists in the registry with id '064258348567'


In [11]:
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

The push refers to repository [064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-processing-container]

3bf68033: Preparing 
463c6a5e: Preparing 
018eda14: Preparing 
6a810ca1: Preparing 
6bb9ffea: Preparing 
663e622b: Preparing 
845af46d: Preparing 
3bf68033: Pushed    5.12kBists latest: digest: sha256:e8b8279167e5045ff35817f45c855897bc0ad91834f08cd084e6d1bbb65bf5a2 size: 2004


In [23]:
!echo $processing_repository_uri

064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-processing-container:latest


# can we now run the image?

In [33]:
role_arn = 'arn:aws:sts::064258348567:assumed-role/AmazonSageMaker-ExecutionRole-20200131T104492/SageMaker'

In [51]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

script_processor = ScriptProcessor(command=['python3'],
                image_uri=processing_repository_uri,
                role=role,
                instance_count=1,
                instance_type='ml.m5.2xlarge')


In [52]:
script_processor.run(code='scripts/congress_pre_process.py',
                     inputs=[ProcessingInput(
                        source='s3://ascsagemaker/JMP_congressional_nmf/raw_data',
                        destination='/opt/ml/processing/input')],
                      outputs=[ProcessingOutput(
                        source='/opt/ml/processing/output',
                        destination='s3://ascsagemaker/JMP_congressional_nmf/dtms')],
                      arguments=['110']
                    )


Job Name:  sagemaker-processing-container-2021-04-26-20-12-13-551
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://ascsagemaker/JMP_congressional_nmf/raw_data', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-064258348567/sagemaker-processing-container-2021-04-26-20-12-13-551/input/code/congress_pre_process.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output-1', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://ascsagemaker/JMP_congressional_nmf/dtms', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
........................../usr/local/lib/python3.7/site-packages/gensim

pre-processed speeches

